In [44]:
import os
import cv2
import os.path as osp
import glob
import xml.etree.ElementTree as ET
from xml.dom import minidom
import tqdm

# Utils

In [13]:
class Rect:
    def __init__(self, x=0, y=0, w=0, h=0):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        
def load_rects(rects_filepath):
    f = open(rects_filepath, 'r')
    rects = []
    rects_count = int(f.readline())
    for j in range(0, rects_count):
        rect_values = str(f.readline()).split(' ')
        rect = Rect(int(rect_values[0]), int(rect_values[1]), int(rect_values[2]), int(rect_values[3]))
        rects.append(rect)
    return rects

# Input

In [38]:
data_dir = '../data/labeled_images/'

# Conversion

In [39]:
annotations = list(glob.glob(osp.join(data_dir, '*.txt')))
len(annotations)

61

In [45]:
images = {osp.basename(p): cv2.imread(osp.splitext(p)[0] + '.jpg').shape for p in tqdm.tqdm_notebook(annotations)}

In [52]:
def convert_to_xml(xml_path, image_path, rects):
    txt_path = osp.basename(osp.splitext(image_path)[0] + '.txt')
    root = ET.Element("annotation")
    root.set('verified', 'yes')
    ET.SubElement(root, "folder").text = 'labeled_images'
    ET.SubElement(root, "filename").text = osp.basename(image_path)
    ET.SubElement(root, "path").text = image_path
    ET.SubElement(ET.SubElement(root, "source"), "database").text = 'Unknown'
    image_size = ET.SubElement(root, "size")
    ET.SubElement(image_size, "width").text = str(images[txt_path][1])
    ET.SubElement(image_size, "height").text = str(images[txt_path][0])
    ET.SubElement(image_size, "depth").text = str(images[txt_path][2])
    ET.SubElement(root, "segmented").text = "0"
    
    for rect in rects:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = "neuron"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"
        bnd_box = ET.SubElement(obj, "bndbox")
        ET.SubElement(bnd_box, "xmin").text = str(rect.x)
        ET.SubElement(bnd_box, "ymin").text = str(rect.y)
        ET.SubElement(bnd_box, "xmax").text = str(rect.x + rect.w)
        ET.SubElement(bnd_box, "ymax").text = str(rect.y + rect.h)
    
    xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="\t")
    with open(xml_path, "w") as f:
        f.write(xmlstr)
        

images_rects = dict()
for image_annotation_path in annotations:
    image_path = osp.splitext(image_annotation_path)[0] + '.jpg'
    xml_path = osp.splitext(image_annotation_path)[0] + '.xml'
    
    rects = load_rects(image_annotation_path)
    images_rects[image_path] = rects
    
    convert_to_xml(xml_path, image_path, rects)